In [1]:
import os

if "notebooks" in os.getcwd():
    # Change the current working directory
    os.chdir("..")

### Test Function Calling

In [4]:
from model.conversation import Conversation
from modules.chatbot import ChatBot

save_questions_description = {
  "name": "save_questions",
  "description": "Set daily journal questions for the user. The questions can be for a Gratitude journal (What are you grateful for?), self-improvement (what charged you with energy, what drained you of energy, what would you like to do more, what would you like to do less), or anything else the user wants as a goal.",
  "parameters": {
    "type": "object",
    "properties": {
      "questions": {
        "type": "array",
        "items": {
          "type": "string"
        },
        "description": "array of strings, representing the questions the user wants to set as their daily questions"
      }
    },
    "required": [
      "questions"
    ]
  }
}

chatbot = ChatBot(function_descriptions=[save_questions_description])
conversation = Conversation(chatbot.system_message)

conversation.add_turn("user", "Can you add the following questions as my daily questions: 1. What made me happy, 2. What made me sad")

response = chatbot.chat(conversation)
print(response)

KeyboardInterrupt: 